In [77]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import numpy as np
from numpy import inf
import cplex
from cplex.exceptions import CplexError
import sys
from collections import Iterable
import pandas as pd
from scipy.io import arff

In [78]:
path = './data/liver.txt'

def ramp_loss(file_path, C = 10):
    
    set_kernel = 'linear'
    set_C = C
    set_dual = False
    set_localimplied = 3
    set_timelimit = 10
    
    f = open(file_path, 'r')
    data, meta = arff.loadarff(f)
    data = np.apply_along_axis(lambda x: x.tolist(), 0, data)

    dim = data.shape[1]
    X = data[:, 0:dim-2].astype(float)
    y = data[:, dim-1].astype(int)
    # Process data
    processLabel(y)
    X = normalize(X)
    X = np.nan_to_num(X)
    # Split to train and test datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1238)
    
    SVM_R(X_train, y_train, X_test, y_test)

In [79]:
def processLabel(y_set):
    for i in range(y_set.shape[0]):
        if y_set[i] == 2:
            y_set[i] = -1

def processData(X_set):
    for i in range(X_set.shape[0]):
        for j in range(X_set.shape[1]):
            X_set[i, j] = (X_set[i, j] - np.mean(X[j])) / 2 * np.std(X[j])

In [80]:
# For square matrices
def matrixToList(mat):
    matList = []
    for i in range(mat.shape[1]):
        matList.append([range(mat.shape[1]), mat[i].tolist()])
    return matList

In [81]:
def flatten(iterable):
    for el in iterable:
        if isinstance(el, Iterable) and not isinstance(el, str): 
            yield from flatten(el)
        else:
            yield el

In [82]:
def setProblemData(p, X_train, y_train, dual = False, C = 10):
    if dual == False:
        print("setting primal problem")
        p.set_problem_name("SVM_HM_IP1")
        p.objective.set_sense(p.objective.sense.minimize)
        # define variable names
        
        my_colnames = [["w" + str(i) for i in range(1, X_train.shape[1] + 1)], ["b"],
                       ["E" + str(i) for i in range(1, X_train.shape[0] + 1)],
                       ["z" + str(i) for i in range(1, X_train.shape[0] + 1)]]
        # add w(i) variables
        p.variables.add(types = [p.variables.type.continuous] * len(my_colnames[0]),
                        names = my_colnames[0], lb=[- cplex.infinity]*len(my_colnames[0]))
        
        qmat = matrixToList(np.identity(X_train.shape[1]))
        
        p.objective.set_quadratic(qmat)
        # add b
        p.variables.add(obj = [0], types = p.variables.type.continuous, names = "b", lb=[- cplex.infinity])
        # add z(i) variables
        p.variables.add(obj = [2*C] * len(my_colnames[3]), types = [p.variables.type.binary] * len(my_colnames[3]), 
                        names = my_colnames[3])
        # add E(i)
        p.variables.add(obj=[C] * len(my_colnames[2]), 
                        types = [p.variables.type.continuous] * len(my_colnames[2]), 
                        names = my_colnames[2],lb=[0] * len(my_colnames[2]), 
                        ub=[2] * len(my_colnames[2]))
        
        coefs = []
        for i in range(X_train.shape[0]):
            coefs.append([y_train[i] * X_train[i], float(y_train[i]), 1.0])
            coefs[i][0] = coefs[i][0].tolist()
        
        wlist = my_colnames[0]
        Elist = my_colnames[2]
        
        
        for n in range(X_train.shape[0]):
            inds = list(flatten([wlist, "b", Elist[n]]))
            fcoefs = list(flatten(coefs[n]))
            lin_exp = cplex.SparsePair(ind=inds, val=fcoefs)
            p.indicator_constraints.add(indvar= my_colnames[3][n], complemented=1,
                                        rhs=1.0, sense='G',
                                        lin_expr=lin_exp)

In [83]:
def predict(p, X_train, y_train, X_test, y_test, Dual=False):
    
    #X_test, y_test
    Test_set = X_test
    label_test = y_test
   
    sol = p.solution
    global test_predicted
    test_predicted = np.zeros(shape=label_test.shape[0])
    
    if Dual == False:
        sol_vals = []
        
        for i in range(X_train.shape[1] + 1):
            sol_vals.append(sol.get_values(i))
        
        w = np.asarray(sol_vals[0:len(sol_vals)-1])
        b = sol_vals[len(sol_vals)-1]
        
        for j in range(Test_set.shape[0]):
            test_predicted[j] = np.sign(np.inner(w, X_test[j]) + b)
    
    TP = np.zeros(shape=label_test.shape[0])
    TN = np.zeros(shape=label_test.shape[0])
    FP = np.zeros(shape=label_test.shape[0])
    FN = np.zeros(shape=label_test.shape[0])
    
    for i in range(label_test.shape[0]):
        if label_test[i] == 1 and test_predicted[i] == 1:
            TP[i] = 1
        elif label_test[i] == 1 and test_predicted[i] == -1:
            FN[i] = 1
        elif label_test[i] == -1 and test_predicted[i] == 1:    
            FP[i] = 1
        elif label_test[i] == -1 and test_predicted[i] == -1:
            TN[i] = 1
    
    Confusion_matrix = [[np.sum(TP), np.sum(FN)], [np.sum(FP), np.sum(TN)]]        
    print("Confusion matrix = ([TP, FN], [FP, TN]) = ", Confusion_matrix)


    Sensitivity = Confusion_matrix[0][0] / (Confusion_matrix[0][0] + Confusion_matrix[0][1])
    Precision = Confusion_matrix[0][0] / (Confusion_matrix[0][0] + Confusion_matrix[1][0])
    Accuracy = (Confusion_matrix[0][0] + Confusion_matrix[1][1]) / (Confusion_matrix[0][0] + 
                Confusion_matrix[1][1] + Confusion_matrix[0][1] + Confusion_matrix[1][0])   
    print("Classifier Accuracy = ", Accuracy )
    print("Precision = ", Precision)
    print("Sensitivity = ", Sensitivity)
           
    return test_predicted

In [84]:
def SVM_R(X_train, y_train, X_test, y_test):
    
    set_localimplied = 3
    set_timelimit = 10
    
    p = cplex.Cplex()
    setProblemData(p, X_train, y_train)
    
    p.parameters.mip.cuts.localimplied.set(set_localimplied)
    p.parameters.timelimit.set(set_timelimit)
    
    print("Solving Ramp Loss SVM primal problem")
    
    p.solve()
    
    #p.write("SVMIP1_HM.lp")

    sol = p.solution
   
    #sol.write("Primal_Solution.lp")

    # solution.get_status() returns an integer code
    print("Solution status = ", sol.get_status(), ":", end=' ')
    
    # the following line prints the corresponding string
    print(sol.status[sol.get_status()])
    print("Solution value  = ", sol.get_objective_value())

    numcols = p.variables.get_num()

    for j in range(numcols):
        if sol.get_values(j) != 0:
            print("Column %d: Value = %10f" % (j, sol.get_values(j)))
    
    predict(p, X_train, y_train, X_test, y_test)

In [85]:
ramp_loss(path)

setting primal problem
Solving Ramp Loss SVM primal problem
CPXPARAM_TimeLimit                               10
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Cuts_LocalImplied                   3
Tried aggregator 1 time.
Reduced MIQP has 207 rows, 627 columns, and 1656 nonzeros.
Reduced MIQP has 207 binaries, 0 generals, 0 SOSs, and 207 indicators.
Reduced MIQP objective Q matrix has 5 nonzeros.
Presolve time = 0.07 sec. (0.68 ticks)
Probing time = 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
Reduced MIQP has 207 rows, 627 columns, and 1656 nonzeros.
Reduced MIQP has 207 binaries, 0 generals, 0 SOSs, and 207 indicators.
Reduced MIQP objective Q matrix has 5 nonzeros.
Presolve time = 0.02 sec. (0.43 ticks)
Probing time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.62 ticks)

        Nodes                                   

Confusion matrix = ([TP, FN], [FP, TN]) =  [[8.0, 51.0], [0.0, 79.0]]
Classifier Accuracy =  0.6304347826086957
Precision =  1.0
Sensitivity =  0.13559322033898305
